In [11]:
import redis
from urllib.parse import urlparse
import pandas as pd
import json

In [3]:
REDIS_URL = 'redis://:p0384551a279fb26f32b9f1d4b24df523826d1d2c09bc1993185b8f202d5144b0@ec2-52-205-73-86.compute-1.amazonaws.com:24589'

In [5]:
parsed_url = urlparse(REDIS_URL)
redis_client = redis.Redis(host=parsed_url.hostname, port=parsed_url.port, password=parsed_url.password)
REDIS_KEY = "mlb_data" 

In [6]:
redis_data = redis_client.get(REDIS_KEY)

In [9]:
df = pd.DataFrame()

In [12]:
data = json.loads(redis_data)

In [16]:
data

[{'id': '168817c6882447f97c2c703ff7f8d926',
  'sport_key': 'baseball_mlb',
  'sport_title': 'MLB',
  'commence_time': '2023-06-27T23:05:00Z',
  'home_team': 'Baltimore Orioles',
  'away_team': 'Cincinnati Reds',
  'bookmakers': [{'key': 'draftkings',
    'title': 'DraftKings',
    'last_update': '2023-06-27T13:50:41Z',
    'markets': [{'key': 'h2h',
      'last_update': '2023-06-27T13:50:41Z',
      'outcomes': [{'name': 'Baltimore Orioles', 'price': -140},
       {'name': 'Cincinnati Reds', 'price': 120}]}]},
   {'key': 'fanduel',
    'title': 'FanDuel',
    'last_update': '2023-06-27T13:51:19Z',
    'markets': [{'key': 'h2h',
      'last_update': '2023-06-27T13:51:19Z',
      'outcomes': [{'name': 'Baltimore Orioles', 'price': -142},
       {'name': 'Cincinnati Reds', 'price': 120}]}]},
   {'key': 'betus',
    'title': 'BetUS',
    'last_update': '2023-06-27T13:50:12Z',
    'markets': [{'key': 'h2h',
      'last_update': '2023-06-27T13:50:12Z',
      'outcomes': [{'name': 'Baltimore 

In [14]:
def preprocess_redis_data(redis_data):
    # Load the data as a JSON object
   

    # Extract and reformat data into a list of dictionaries
    extracted_data = []
    for game in redis_data:
        row = {
            'game_id': game['id'],
            'home_team': game['home_team'],
            'away_team': game['away_team'],
            'commence_time': game['commence_time'],
            'game_date': None,  # This needs to be generated from 'commence_time' or provided separately
            'season': None,  # This information is not provided in the sample data and needs to be added
            'game_type': None  # This information is not provided in the sample data and needs to be added
        }
        for bookmaker in game['bookmakers']:
            bookmaker_key = bookmaker['key']
            for market in bookmaker['markets']:
                if market['key'] == 'h2h':
                    for outcome in market['outcomes']:
                        if outcome['name'] == row['home_team']:
                            row[f'{bookmaker_key}_home'] = outcome['price']
                        elif outcome['name'] == row['away_team']:
                            row[f'{bookmaker_key}_away'] = outcome['price']

        extracted_data.append(row)
    df = pd.DataFrame(extracted_data)
    for idx, row in df.iterrows():
        home_columns = [col for col in df.columns if col.endswith('_home')]
        away_columns = [col for col in df.columns if col.endswith('_away')]

        home_median = np.nanmedian(row[home_columns])
        away_median = np.nanmedian(row[away_columns])

        df.loc[idx, home_columns] = row[home_columns].fillna(home_median)
        df.loc[idx, away_columns] = row[away_columns].fillna(away_median)

    required_columns = ['lowvig_home', 'lowvig_away', 'betonlineag_home', 'betonlineag_away',
       'unibet_home', 'unibet_away', 'draftkings_home', 'draftkings_away',
       'pointsbetus_home', 'pointsbetus_away', 'gtbets_home', 'gtbets_away',
       'mybookieag_home', 'mybookieag_away', 'bovada_home', 'bovada_away',
       'fanduel_home', 'fanduel_away', 'intertops_home', 'intertops_away',
       'williamhill_us_home', 'williamhill_us_away', 'betrivers_home',
       'betrivers_away', 'betmgm_home', 'betmgm_away', 'sugarhouse_home',
       'sugarhouse_away', 'foxbet_home', 'foxbet_away', 'barstool_home',
       'barstool_away', 'twinspires_home', 'twinspires_away', 'betus_home',
       'betus_away', 'wynnbet_home', 'wynnbet_away', 'circasports_home',
       'circasports_away', 'superbook_home', 'superbook_away',
       'unibet_us_home', 'unibet_us_away']

    for col in required_columns:
        if col not in df.columns:
            df[col] = np.nan
    
    # Fill in missing values in each row with the median of the row
    
    # Fill in missing values in each row with the median of the row
    #df = df.apply(lambda x: x[].fillna(x.median()), axis=1)
    home_columns = [col for col in required_columns if col.endswith('_home')]
    away_columns = [col for col in required_columns if col.endswith('_away')]
    df[home_columns] = df[home_columns].apply(lambda x: x.fillna(x.median()), axis=1)
    df[away_columns] = df[away_columns].apply(lambda x: x.fillna(x.median()), axis=1)


    
    return df

In [15]:
preprocess_redis_data(redis_data)

TypeError: 'int' object is not subscriptable

In [7]:
redis_data

b'[{"id": "168817c6882447f97c2c703ff7f8d926", "sport_key": "baseball_mlb", "sport_title": "MLB", "commence_time": "2023-06-27T23:05:00Z", "home_team": "Baltimore Orioles", "away_team": "Cincinnati Reds", "bookmakers": [{"key": "draftkings", "title": "DraftKings", "last_update": "2023-06-27T13:50:41Z", "markets": [{"key": "h2h", "last_update": "2023-06-27T13:50:41Z", "outcomes": [{"name": "Baltimore Orioles", "price": -140}, {"name": "Cincinnati Reds", "price": 120}]}]}, {"key": "fanduel", "title": "FanDuel", "last_update": "2023-06-27T13:51:19Z", "markets": [{"key": "h2h", "last_update": "2023-06-27T13:51:19Z", "outcomes": [{"name": "Baltimore Orioles", "price": -142}, {"name": "Cincinnati Reds", "price": 120}]}]}, {"key": "betus", "title": "BetUS", "last_update": "2023-06-27T13:50:12Z", "markets": [{"key": "h2h", "last_update": "2023-06-27T13:50:12Z", "outcomes": [{"name": "Baltimore Orioles", "price": -140}, {"name": "Cincinnati Reds", "price": 125}]}]}, {"key": "lowvig", "title": "L